In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

import sys
sys.path.insert(0, '../helpers/')
from plot_graphs import plot_loss, plot_acc

In [3]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

train = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

train = train.prefetch(buffer_size=tf.data.AUTOTUNE).cache()
val = val.prefetch(buffer_size=tf.data.AUTOTUNE).cache()
test = test.prefetch(buffer_size=tf.data.AUTOTUNE).cache()

Found 3840 files belonging to 6 classes.
Found 480 files belonging to 6 classes.
Found 480 files belonging to 6 classes.


In [4]:
train = train.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
val = val.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
test = test.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))

In [5]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    pooling='avg',
    classes=6,
    classifier_activation="softmax"
)

model.compile(optimizer=keras.optimizers.RMSprop(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [6]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../models/6_sota_vgg16.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history = model.fit(
  train,
  epochs=20,
  callbacks=callbacks_list,
  validation_data=val
)

2022-08-29 10:28:42.757638: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20


2022-08-29 10:28:43.321145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - ETA: 0s - loss: 1.3068 - accuracy: 0.5057

2022-08-29 10:29:45.175614: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - 65s 526ms/step - loss: 1.3068 - accuracy: 0.5057 - val_loss: 0.6190 - val_accuracy: 0.7646
Epoch 2/20
120/120 [==============================] - 64s 536ms/step - loss: 0.5516 - accuracy: 0.8036 - val_loss: 0.4198 - val_accuracy: 0.8771
Epoch 3/20
120/120 [==============================] - 66s 551ms/step - loss: 0.3233 - accuracy: 0.8805 - val_loss: 0.3328 - val_accuracy: 0.8938
Epoch 4/20
120/120 [==============================] - 69s 579ms/step - loss: 0.1993 - accuracy: 0.9349 - val_loss: 0.2596 - val_accuracy: 0.9125
Epoch 5/20
120/120 [==============================] - 68s 566ms/step - loss: 0.1661 - accuracy: 0.9464 - val_loss: 0.2137 - val_accuracy: 0.9438
Epoch 6/20
120/120 [==============================] - 77s 638ms/step - loss: 0.1355 - accuracy: 0.9555 - val_loss: 0.2230 - val_accuracy: 0.9250
Epoch 7/20
120/120 [==============================] - 88s 734ms/step - loss: 0.1226 - accuracy: 0.9654 - val_loss: 0.2231 - val_accuracy: 0.9

In [7]:
load_model = tf.keras.models.load_model("../models/6_sota_vgg16.h5")
load_model.evaluate(test)

2022-08-29 10:41:55.517091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 2s 124ms/step - loss: 0.1819 - accuracy: 0.9396


[0.18189123272895813, 0.9395833611488342]